You have different tasks to finish within the day. 7 am is the earliest time and 10 pm is the latest time of the day to finish all the tasks. Since each time block is 30', there are (15 hours)*(2 blocks/hour)= 30 blocks. Each task takes different amount of time to complete and different important score. Given that there are scheduled meetings during the day that you cannot assign the tasks.  How do you assign the tasks to maximize the productivty? Note: Since there is limited time per day, you don't need to assign every task in one day, the goal is not to fit every task but to maximize the total score

Input parameters: number of tasks (n), number of time blocks (B), important score of a task (s[i]), time for a task (t[i]), availability of a block (a[b]), with a[b]=0 if not available and a[b]=1 if available

Decision Variables: Whether or not to assign the task in a specific block of time

Constraints:
-One task per block
-Time performing all tasks should be less than the time available
-Do not need to be consecutive

In [ ]:
!pip3 install pandas
!pip3 install pulp
!pip3 install tox
!pip3 install matplotlib
!pip3 install openpyxl

In [ ]:
import pandas as pd
import pulp
import tox
import matplotlib
import openpyxl

In [ ]:
tasks = pd.read_excel('data/Tasks.xlsx', 'Tasks')
tasks

In [ ]:
schedule = pd.read_excel('data/Tasks.xlsx','Schedule',usecols="B",header=None)
schedule


In [ ]:
s = list(tasks['Important score (1-5)'])

d = list(tasks['Num of blocks'])

b = list(schedule.iloc[:,0])


In [ ]:
B = len(b)
n = len(s)
M = 10000000

In [ ]:
#Time blocks available
A = sum(b)
A

In [ ]:
from pulp import *

In [ ]:
prob = LpProblem("Schedule_Tasks",LpMaximize)

In [ ]:
#Define variable
y = LpVariable.dicts('Block', [(i,t) for i in range(n) for t in range(B)],
                    cat='Binary')
start = LpVariable.dicts('Startime', [i for i in range(n)], cat='Integer', lowBound=1, upBound=B)

x = LpVariable.dicts('Precedence {i}_{j+1}',[(i,j) for i in range(n) for j in range(n)], cat="Binary")

first = LpVariable.dicts('First Task', [i for i in range(n)], cat= 'Binary')

last = LpVariable.dicts('Last Task', [i for i in range(n)], cat = 'Binary')


In [ ]:
y

In [ ]:
#Define objective

prob += lpSum(s[i]*b[t]*y[(i,t)] for i in range(n) for t in range(B))

In [ ]:
#Define constraints

#Sum of the time bocks of the assigned tasks should be not greater than the number of available time blocks
prob += lpSum(y[(i,t)] for i in range(n) for t in range(B)) <= A

#Total number of time blocks assigned for each task should be less than or equal to the time needed to finish the task
for i in range(n):
    prob += lpSum(y[(i,t)] for t in range(B)) <= d[i]

#No more than one task each block
for t in range(B):
    prob += lpSum(y[(i,t)] for i in range(n)) <= 1

#One first task
prob += lpSum(first[i] for i in range(n)) == 1

#One last task
prob += lpSum(last[i] for i in range(n)) == 1


#Task i the last task or it has a successor j

for i in range(n):
    prob += last[i] + lpSum(x[(i,j)] for j in range(n) if j!=i) == 1

#Task j is the first task or it has a precedence i

for j in range(n):
    prob += first[j] + lpSum(x[(i,j)]for i in range(n) if i!=j) == 1

#Condition is applied when task j follows task i
for t in range(B):
    for i in range(n):
        for j in range(n):
            if i!= j:
                prob += start[i] + d[i] - M*(1 - x[(i,j)]) - M*(1-y[(i,t)]) - M*(1-y[(j,t)]) <= start[j]




In [ ]:
LpSolverDefault.msg = 1

In [ ]:
prob.solve()

In [ ]:
print("Assignment accomplished!")
for i in range(n):
    for t in range(B):
        if y[(i,t)].varValue >= 0.5:
            print("task {} block {} start {} first {} last {} precedence {}".format(i,t, start[i].varValue, first[i].varValue, last[i].varValue, x[i,j].varValue))


In [ ]:
pulp.value(prob.objective)

In [ ]:
prob.writeLP('task_problem.lp')

Questions:
-Is it necessary to complete one task in one day when get started?
-How to make the tasks continuous once get started but doesn't need to finish on the same day if the time does not allow?
    +precedence constraints (Single Machine Scheduling): One task cannot start until another task starts
    +No overlap constraints: One task at a time

Observations:
-The tasks with highest scores would be attempted to fit into the schedule until either time blocks are ran out out the task is finished
